In [18]:
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv
import json


In [19]:
#Data Extraction
data = gpd.read_file("/Users/ACER/Desktop/VIP_IDI/atl_bg.geojson")
polygons=[ i for i in data["geometry"]]#extracting polygons  from the geojson file
areas = [i for i in data["ALAND"]]#extracting area from the geojson file


In [20]:
# Initialize empty lists for calculation results
numerator = []
intersection_sums = []

# Iterate through polygons and their corresponding areas
for polygon, area in zip(polygons, areas):
    intersection_factor_sum = 0

    # Create a street network graph from the current polygon
    graph = ox.graph_from_polygon(polygon, network_type='walk') 

    # Calculate intersection factor sum for the polygon's graph
    for node in graph.nodes():
        intersection_factor_sum += len(list(graph.neighbors(node)))  # Count neighbors of each node

    # Store results    
    intersection_sums.append(intersection_factor_sum) 
    numerator.append(intersection_factor_sum/area)  # Calculate metric, normalized by area

# Find the maximum value in 'numerator'
maxi = max(numerator)

# Calculate the IDI 
IDI = [value / maxi for value in numerator]  # Normalize values by the maximum

In [21]:
print(IDI)

500


In [ ]:

# Create features list
features = []
for i in range(len(polygons)):
  feature = {
      "type": "Feature",
      "geometry": {
          "type": "Polygon",
          "coordinates": polygons[i] 
      },
      "properties": {
          "Area": areas[i],
          "Intersection_sums": intersection_sums[i],
          "Numerator": numerator[i],
          "IDI": IDI[i]
      }
  }
  features.append(feature)

# Create GeoJSON object
geojson = {
  "type": "FeatureCollection",
  "features": features
}

# Convert to JSON string
json_string = json.dumps(geojson, indent=4)

# Write to file
with open('Atlanta_IDI_data.geojson', 'w') as outfile:
  outfile.write(json_string)

In [ ]:
# Create and write to the CSV file
with open('Atlanta_IDI_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Polygon', 'Area', 'Intersection_sums','Numerator','IDI'])  # Header row

    # Use zip to combine lists into rows
    for polygon, area,sum,num,val in zip(polygons, areas, intersection_sums,numerator,IDI):
        writer.writerow([polygon, area,sum,num,val])